In [64]:
# pip install jupyterthemes

In [65]:
# !jt -l

In [66]:
# !jt -t monokai

In [67]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras import metrics

# cls sentance sep cls means begining sep seperats two differnt sentance

In [68]:
import pandas as pd 
df = pd.read_csv('dataset/spamdetection.csv',sep='\t',names=['label','message']).dropna()
df.head(10)

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...
6,ham,Even my brother is not like to speak with me. ...
7,ham,As per your request 'Melle Melle (Oru Minnamin...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...


In [69]:
v = df['label'].value_counts()
ham = df.loc[df['label'] == 'ham'][0:150]
spam = df.loc[df['label']== 'spam'][0:150]
df = pd.concat([ham,spam]).sample(frac=1)


In [70]:
x = list(df['message'])
y = list(df['label'])


In [71]:
y = list(pd.get_dummies(y,drop_first=True) ['spam']) #take spam column then convert to list


In [72]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.20,random_state=0)

In [73]:
encoder_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4'
preprocess_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'
bert_preprocess = hub.KerasLayer(preprocess_url)
bert_encoder = hub.KerasLayer(encoder_url)

In [74]:
#bert layers
text_input = tf.keras.layers.Input(shape=(),dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
output = bert_encoder(preprocessed_text)

In [75]:
l = Dropout(0.1, name= 'dropout')(output['pooled_output'])
l = Dense(1,activation='sigmoid',name = 'output')(l)

In [76]:
#construct a final model
model = tf.keras.Model(inputs=[text_input],outputs=[l])
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text (InputLayer)               [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer_2 (KerasLayer)      {'input_type_ids': ( 0           text[0][0]                       
__________________________________________________________________________________________________
keras_layer_3 (KerasLayer)      {'pooled_output': (N 109482241   keras_layer_2[0][0]              
                                                                 keras_layer_2[0][1]              
                                                                 keras_layer_2[0][2]              
____________________________________________________________________________________________

In [88]:
METRICS = [
#   metrics.BinaryAccuracy(name='accuracy'),
   'accuracy',
    metrics.Precision(name='precision'),
    metrics.Recall(name='recall')
]
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=METRICS)

In [90]:
model.fit(X_train,y_train,validation_data=(X_test, y_test),epochs=2)

Epoch 1/2
8/8 [==============================] - 205s 26s/step - loss: 0.6090 - accuracy: 0.6542 - precision: 0.6977 - recall: 0.5128 - val_loss: 0.5614 - val_accuracy: 0.7500 - val_precision: 0.6875 - val_recall: 1.0000
Epoch 2/2
8/8 [==============================] - 213s 27s/step - loss: 0.5905 - accuracy: 0.7208 - precision: 0.7119 - recall: 0.7179 - val_loss: 0.5516 - val_accuracy: 0.8333 - val_precision: 0.9259 - val_recall: 0.7576


In [91]:
history = model.history.history
print(history.keys())

dict_keys(['loss', 'accuracy', 'precision', 'recall', 'val_loss', 'val_accuracy', 'val_precision', 'val_recall'])


In [92]:
df = pd.DataFrame(model.history.history)
df.to_csv('dataset/bartModelHistory.csv')

In [129]:
reviews = 'SIX chances to win CASH! From 100 to 20,000 pounds txt> CSH11 and send to 87575. Cost 150p/day, 6days, 16+ TsandCs apply Reply HL 4 info'

In [130]:
pred = model.predict([reviews])
pred


array([[0.5618419]], dtype=float32)

In [137]:
predlis = round(pred.tolist()[0][0])
value = predlis


In [135]:
#using numpy
#by this way we can predict multiple comment
import numpy as np
finalPred = np.where(predlis>0.5,1,0) #where predlis value> put 1 other wise put 0
value = finalPred.tolist()


In [138]:
if(value == 1):
    print('Spam')
else:
    print('ham')
    

Spam
